In [1]:
import pandas as pd
import numpy as np

# Tablas Pivote

Es muy común la agrupación de información en formato 'stack' donde tenemos filas de datos que demuestran una correlación entre dos sets de valores.

Las tablas pivote son una forma de re-ordenar los datos en una estructura tabular donde podemos agrupar los valores convirtiendo las tuplas entre valores numéricos.

En este caso de ejemplo, crearemos un dataset de prueba con la diversidad de restaurantes en varias ciudades centroamericanas. En esta tendremos dos columnas, una donde describe cada ciudad y la otra con la variedad de cocina disponible en cada una.

In [14]:
data_restaurantes = {
    'ciudades': ['Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','San José','San José','San José','San José','San José','San Salvador','San Salvador','San Salvador'],
    'restaurantes': ['Chapina','Chapina','China','Thai','Italiana','Chapina','Italiana','China','Tica','Chapina','Tica','Tica','Italiana','China'],
    'estrellas_michelin': [0,1,0,5,3,1,2,2,4,0,4,3,2,3]
}

restaurantes_dataframe_pares = pd.DataFrame(data_restaurantes)

In [15]:
restaurantes_dataframe_pares

,ciudades,estrellas_michelin,restaurantes
0,Guatemala,0,Chapina
1,Guatemala,1,Chapina
2,Guatemala,0,China
3,Guatemala,5,Thai
4,Guatemala,3,Italiana
5,Guatemala,1,Chapina
6,San José,2,Italiana
7,San José,2,China
8,San José,4,Tica
9,San José,0,Chapina


Podemos ver entonces este listado de valores, tupla por tupla. Que tal si queremos contar la presencia de cada tipo de cocina en cada region. Probemos utilizando entonces el comando [DataFrame.pivot_table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html) de Pandas.

Este pide unos cuantos argumentos que podemos ver en la documentación. Unos cuantos son obvios, como el definir las filas y columnas que esperamos de la tabla objetivo.
Sin embargo, lo más notable es que ya que los tipos de datos que estamos utilizando no son numéricos, es necesario que definamos una funcion de agrupación que nos permita contar la cantidad de instancias de cada combinación.

In [16]:
def funcion_agrupacion(elemento):
    return True

agrupacion_culinaria = restaurantes_dataframe_pares.pivot_table(index=["ciudades"], columns="restaurantes", aggfunc=funcion_agrupacion, fill_value=0)

In [17]:
agrupacion_culinaria

estrellas_michelin                           
restaurantes            Chapina China Italiana  Thai  Tica
ciudades                                                  
Guatemala                  True  True     True  True     0
San José                   True  True     True     0  True
San Salvador                  0  True     True     0  True

Hmm, esto ya se ve con la forma que queremos que tenga, sin embargo solo nos muestra la presencia o ausencia de algun tipo de cocina. Que tal si hacemos una mejor función de agrupación.

In [18]:
def funcion_agrupacion(elemento):
    '''Contemos cuantas instancias de cada tupla existen.'''
    ## El comando len(iterable) cuenta la cantidad de elementos que tiene el objeto iterable que le pasemos
    ## los elementos iterables pueden ser listas normales, Series de NumPy o Pandas, o diccionarios y otros tipos de datos.
    return len(elemento)
agrupacion_culinaria = restaurantes_dataframe_pares.pivot_table(index=["ciudades"], columns="restaurantes", aggfunc=lambda x: funcion_agrupacion(x), fill_value=0)

In [19]:
agrupacion_culinaria

estrellas_michelin                         
restaurantes            Chapina China Italiana Thai Tica
ciudades                                                
Guatemala                     3     1        1    1    0
San José                      1     1        1    0    2
San Salvador                  0     1        1    0    1

Genial! Ahora ya tenemos agrupadas estas de una forma coherente. Ahora ya podemos seguir manipulando y editando estos datos.
Pero que tal si hacemos un poco mas simple esta llamada, al final nuestra funcion de agrupacion lo unico que hace es llamar len sobre el objeto, que tal si lo hacemos un poco más simple. 

In [8]:
agrupacion_culinaria = restaurantes_dataframe_pares.pivot_table(
    index=["ciudades"], 
    columns="restaurantes", 
    aggfunc=len, ## Enviamos directamente la función de agrupación.
    fill_value=0)

In [20]:
agrupacion_culinaria.query('ciudades == ["Guatemala"]')

estrellas_michelin                         
restaurantes            Chapina China Italiana Thai Tica
ciudades                                                
Guatemala                     3     1        1    1    0

In [21]:
## Con el argumento Margins, Panda calcula los valores sumados de los totales por agrupación.
agrupacion_culinaria_m = restaurantes_dataframe_pares.pivot_table(index=["ciudades"], columns="restaurantes", aggfunc=lambda x: len(x), fill_value=0, margins=True)
agrupacion_culinaria_m

estrellas_michelin                               
restaurantes            Chapina China Italiana Thai Tica   All
ciudades                                                      
Guatemala                   3.0   1.0      1.0  1.0  0.0   6.0
San José                    1.0   1.0      1.0  0.0  2.0   5.0
San Salvador                0.0   1.0      1.0  0.0  1.0   3.0
All                         4.0   3.0      3.0  1.0  3.0  14.0

## Múltiples Valores y Múltiples Indices
¿Que tal si tenemos datos que tienen una estructura Jerárquica inherente? Podemos utilizar la mísma forma de multi indexación que vimos en el seminario pasado, lo importante es que a la hora de la definición del índice, Pandas es capaz de manipularlos e inteligentemente ordenar los niveles acorde.

In [30]:
agrupacion_culinaria_promedio_estrellas = restaurantes_dataframe_pares.pivot_table(
    index=["ciudades"], 
    values=["restaurantes", "estrellas_michelin"], 
    aggfunc={"restaurantes":len,"estrellas_michelin":np.mean},
    fill_value=0)
agrupacion_culinaria_promedio_estrellas

,estrellas_michelin,restaurantes
ciudades,,
Guatemala,1.666667,6
San José,2.400000,5
San Salvador,2.666667,3


,estrellas_michelin,restaurantes
ciudades,,
Guatemala,1.666667,6
San José,2.400000,5
San Salvador,2.666667,3


In [45]:
agrupacion_culinaria_por_estrellas = restaurantes_dataframe_pares.pivot_table(
    index=["ciudades"], 
    values=["restaurantes", "estrellas_michelin"], 
    columns=["restaurantes", "estrellas_michelin"],
    aggfunc={"restaurantes":len,"estrellas_michelin":np.mean},
    fill_value=0)
agrupacion_culinaria_estrellas

estrellas_michelin                                          \
restaurantes                  Chapina    China       Italiana    Thai Tica      
estrellas_michelin                  0  1     0  2  3        2  3    5    3  4   
ciudades                                                                        
Guatemala                           0  1     0  0  0        0  3    5    0  0   
San José                            0  0     0  2  0        2  0    0    0  4   
San Salvador                        0  0     0  0  3        2  0    0    3  0   

                   restaurantes                                          
restaurantes            Chapina    China       Italiana    Thai Tica     
estrellas_michelin            0  1     0  2  3        2  3    5    3  4  
ciudades                                                                 
Guatemala                     1  2     1  0  0        0  1    1    0  0  
San José                      1  0     0  1  0        1  0    0    0  2  
San Salvador                  0  0     0  0  1        1  0    0    1  0

estrellas_michelin                                          \
restaurantes                  Chapina    China       Italiana    Thai Tica      
estrellas_michelin                  0  1     0  2  3        2  3    5    3  4   
ciudades                                                                        
Guatemala                           0  1     0  0  0        0  3    5    0  0   
San José                            0  0     0  2  0        2  0    0    0  4   
San Salvador                        0  0     0  0  3        2  0    0    3  0   

                   restaurantes                                          
restaurantes            Chapina    China       Italiana    Thai Tica     
estrellas_michelin            0  1     0  2  3        2  3    5    3  4  
ciudades                                                                 
Guatemala                     1  2     1  0  0        0  1    1    0  0  
San José                      1  0     0  1  0        1  0    0    0  2  
San Salvador                  0  0     0  0  1        1  0    0    1  0